# Tema 3: Tutorial sobre Sparklyr

Sparklyr is la interfaz R para Apache Spark, la tecnología de procesamiento Big Data más extendida hoy en día. Permite utilizar dplyr sobre unha plataforma Spark, y aprovechar sus capacidades de procesamiento masivo de datos y Machine Learning en particular (MLlib).

In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# libraries we'll need

library(dplyr)

# No more basic plots!
# https://www.publichealth.columbia.edu/sites/default/files/media/fdawg_ggplot2.html
library(ggplot2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



character(0)

In [2]:
library(sparklyr)
spark_install(version = "3.0.0")

# Se utiliza la versión local como desarrollo, en producción utilizaría un clúster
sc <- spark_connect(master = "local")


Attaching package: ‘sparklyr’


The following object is masked from ‘package:purrr’:

    invoke




In [3]:
sc

$master
[1] "local[4]"

$method
[1] "shell"

$app_name
[1] "sparklyr"

$config
$config$spark.env.SPARK_LOCAL_IP.local
[1] "127.0.0.1"

$config$sparklyr.connect.csv.embedded
[1] "^1.*"

$config$spark.sql.legacy.utcTimestampFunc.enabled
[1] TRUE

$config$sparklyr.connect.cores.local
[1] 4

$config$spark.sql.shuffle.partitions.local
[1] 4

$config$sparklyr.shell.name
[1] "sparklyr"

$config$`sparklyr.shell.driver-memory`
[1] "2g"

attr(,"config")
[1] "default"
attr(,"file")
[1] "/usr/local/lib/R/site-library/sparklyr/conf/config-template.yml"

$state
<environment: 0x56520bcec368>

$extensions
$extensions$jars
character(0)

$extensions$packages
character(0)

$extensions$initializers
list()

$extensions$catalog_jars
character(0)

$extensions$repositories
character(0)


$spark_home
[1] "/root/spark/spark-3.0.0-bin-hadoop3.2"

$backend
A connection with                              
description "->localhost:8881"
class       "sockconn"        
mode        "wb"              
text        "binar

In [4]:
mtcars_tbl <- sdf_copy_to(sc, mtcars, name = "mtcars_tbl", overwrite = TRUE)

partitions <- mtcars_tbl %>%
  sdf_random_split(training = 0.7, test = 0.3, seed = 1111)

mtcars_training <- partitions$training
mtcars_test <- partitions$test

# for multiclass classification
rf_model <- mtcars_training %>%
  ml_random_forest(cyl ~ ., type = "classification")

pred <- ml_predict(rf_model, mtcars_test)

ml_multiclass_classification_evaluator(pred)

[1] 1

In [13]:
mtcars_training

# Source: spark<?> [?? x 11]
     mpg   cyl  disp    hp  drat    wt  qsec    vs    am  gear  carb
   <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1  10.4     8  460    215  3     5.42  17.8     0     0     3     4
 2  13.3     8  350    245  3.73  3.84  15.4     0     0     3     4
 3  14.3     8  360    245  3.21  3.57  15.8     0     0     3     4
 4  14.7     8  440    230  3.23  5.34  17.4     0     0     3     4
 5  15.2     8  276.   180  3.07  3.78  18       0     0     3     3
 6  15.5     8  318    150  2.76  3.52  16.9     0     0     3     2
 7  15.8     8  351    264  4.22  3.17  14.5     0     1     5     4
 8  16.4     8  276.   180  3.07  4.07  17.4     0     0     3     3
 9  17.3     8  276.   180  3.07  3.73  17.6     0     0     3     3
10  18.1     6  225    105  2.76  3.46  20.2     1     0     3     1
# … with more rows

In [14]:
pred

# Source: spark<?> [?? x 18]
    mpg   cyl  disp    hp  drat    wt  qsec    vs    am  gear  carb features
  <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <list>  
1  10.4     8 472     205  2.93  5.25  18.0     0     0     3     4 <dbl [2…
2  15       8 301     335  3.54  3.57  14.6     0     1     5     8 <dbl [2…
3  15.2     8 304     150  3.15  3.44  17.3     0     0     3     2 <dbl [2…
4  17.8     6 168.    123  3.92  3.44  18.9     1     0     4     4 <dbl [2…
5  19.2     8 400     175  3.08  3.84  17.0     0     0     3     2 <dbl [2…
6  21       6 160     110  3.9   2.62  16.5     0     1     4     4 <dbl [2…
7  27.3     4  79      66  4.08  1.94  18.9     1     1     4     1 <dbl [2…
8  32.4     4  78.7    66  4.08  2.2   19.5     1     1     4     1 <dbl [2…
# … with 6 more variables: label <dbl>, rawPrediction <list>,
#   probability <list>, prediction <dbl>, probability_0 <dbl>,
#   probability_1 <dbl>

In [15]:
pred %>% select(cyl,prediction)

# Source: spark<?> [?? x 2]
    cyl prediction
  <dbl>      <dbl>
1     8          0
2     8          1
3     8          0
4     6          1
5     8          0
6     6          1
7     4          1
8     4          1

In [5]:
# for regression
rf_model <- mtcars_training %>%
  ml_random_forest(cyl ~ ., type = "regression")

pred <- ml_predict(rf_model, mtcars_test)

ml_regression_evaluator(pred, label_col = "cyl")

[1] 0.4444097

In [9]:
pred

# Source: spark<?> [?? x 18]
    mpg   cyl  disp    hp  drat    wt  qsec    vs    am  gear  carb features
  <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <list>  
1  10.4     8 472     205  2.93  5.25  18.0     0     0     3     4 <dbl [2…
2  15       8 301     335  3.54  3.57  14.6     0     1     5     8 <dbl [2…
3  15.2     8 304     150  3.15  3.44  17.3     0     0     3     2 <dbl [2…
4  17.8     6 168.    123  3.92  3.44  18.9     1     0     4     4 <dbl [2…
5  19.2     8 400     175  3.08  3.84  17.0     0     0     3     2 <dbl [2…
6  21       6 160     110  3.9   2.62  16.5     0     1     4     4 <dbl [2…
7  27.3     4  79      66  4.08  1.94  18.9     1     1     4     1 <dbl [2…
8  32.4     4  78.7    66  4.08  2.2   19.5     1     1     4     1 <dbl [2…
# … with 6 more variables: label <dbl>, rawPrediction <list>,
#   probability <list>, prediction <dbl>, probability_0 <dbl>,
#   probability_1 <dbl>

In [10]:
pred %>% select(mpg,cyl,prediction, label, rawPrediction, probability)

# Source: spark<?> [?? x 6]
    mpg   cyl prediction label rawPrediction probability
  <dbl> <dbl>      <dbl> <dbl> <list>        <list>     
1  10.4     8          0     0 <dbl [2]>     <dbl [2]>  
2  15       8          1     1 <dbl [2]>     <dbl [2]>  
3  15.2     8          0     0 <dbl [2]>     <dbl [2]>  
4  17.8     6          1     0 <dbl [2]>     <dbl [2]>  
5  19.2     8          0     0 <dbl [2]>     <dbl [2]>  
6  21       6          1     1 <dbl [2]>     <dbl [2]>  
7  27.3     4          1     1 <dbl [2]>     <dbl [2]>  
8  32.4     4          1     1 <dbl [2]>     <dbl [2]>  

In [6]:
# for binary classification
rf_model <- mtcars_training %>%
  ml_random_forest(am ~ gear + carb, type = "classification")

pred <- ml_predict(rf_model, mtcars_test)

ml_binary_classification_evaluator(pred)

## End(Not run)

[1] 0.96875

# # Machine Learning


Sparkly nos permite hacer uso de algoritmo de aprendizaje máquina (Machine Learning). Se trata de programar flujos de trabajo a alto nivel sobre dataframes.



# # Análisis explotario de datos de Iris 

Vamos a explorar el dataset **iris**

In [ ]:
#iris
iris %>% head(n = 6)
iris %>% tail(n = 6)

In [ ]:
str(iris)

In [ ]:
summary(iris)

In [ ]:
scatter <- ggplot(data=iris, aes(x = Sepal.Length, y = Sepal.Width)) 
scatter + geom_point(aes(color=Species, shape=Species)) +
  xlab("Sepal Length") +  ylab("Sepal Width") +
  ggtitle("Sepal Length-Width")

In [ ]:
boxplot(Sepal.Length~Species,data=iris, 
        xlab="Species", ylab="Sepal Length", main="Iris Boxplot")

In [ ]:
box <- ggplot(data=iris, aes(x=Species, y=Sepal.Length))
box + geom_boxplot(aes(fill=Species)) + 
  ylab("Sepal Length") + ggtitle("Iris Boxplot") +
  stat_summary(fun=mean, geom="point", shape=5, size=4) 

In [ ]:
scatter <- ggplot(data=iris, aes(x = Petal.Width, y = Petal.Length)) 
scatter + geom_point(aes(color=Species, shape=Species)) +
  xlab("Petal Width") +  ylab("Petal Length") +
  ggtitle("Petal Length-Width")

In [ ]:
ggplot(iris, aes(x = Petal.Length, y = Petal.Width)) + 
  geom_point() +
  stat_smooth(method = "lm", col = "red")

# # K-means con datos de Iris 

Con el dataset **iris** vamos a aplicar una técnica de clustering, k-means, para clasificación de nuestros datos.

In [ ]:
iris_tbl <- copy_to(sc, iris, "iris", overwrite = TRUE)
iris_tbl

In [ ]:
# https://spark.rstudio.com/reference/ml_kmeans
# La distancia se usa sólo en el score de silhouette 
# donde se puede usar consine o squaredEuclidean

kmeans_model <- iris_tbl %>% 
ml_kmeans(k = 3, features = c("Petal_Length", "Petal_Width"))
kmeans_model

In [ ]:
# predict the associated class
predicted <- ml_predict(kmeans_model, iris_tbl)  %>% collect
#predicted
table(predicted$Species, predicted$prediction)

In [ ]:
# plot cluster membership
ml_predict(kmeans_model) %>%
  collect() %>%
  ggplot(aes(Petal_Length, Petal_Width)) +
  geom_point(aes(Petal_Width, Petal_Length, col = factor(prediction)),
             size = 2, alpha = 0.5) + 
  geom_point(data = kmeans_model$centers, aes(Petal_Width, Petal_Length),
             col = scales::muted(c("blue", "red", "green")),
             pch = 'x', size = 12) +
  scale_color_discrete(name = "Predicted Cluster",
                       labels = paste("Cluster", 1:3)) +
  labs(
    x = "Petal Width",
    y = "Petal Length",
    title = "K-Means Clustering",
    subtitle = "Use Spark.ML to predict cluster membership with the iris dataset."
  )

In [ ]:
iris_tbl%>%
  select(Petal_Width, Petal_Length)

In [ ]:
lm_model

In [ ]:
summary(iris)

In [ ]:
iris_tbl %>%
  ft_bucketizer(input_col  = "Sepal_Length",
                output_col = "Sepal_Length_bucket",
                splits     = c(4, 5.10001, 5.80001, 6.40001, 8)) %>%
  select(Sepal_Length, Sepal_Length_bucket, Species)



In [ ]:
iris_tbl %>%
  ft_bucketizer(input_col  = "Petal_Length",
                output_col = "Petal_Length_bucket",
                splits     = c(1.0, 2, 5, 7)) %>%
  select(Petal_Length, Petal_Length_bucket, Species)

In [ ]:
iris_tbl_df <- iris_tbl %>%
  ft_bucketizer(input_col  = "Petal_Length",
                output_col = "Petal_Length_bucket",
                splits     = c(0.9, 2.5, 4.8, 7.5)) %>%
  select(Petal_Length, Petal_Length_bucket, Species) %>% collect()
iris_tbl_df %>% tail(n=10)

In [ ]:
# Para hacer un modelo predictivo es preciso discretizar estos valores, agrupándolos en conjuntos.
iris_tbl_bucketized <- iris_tbl %>%
  ft_bucketizer(input_col  = "Petal_Length",
                output_col = "Petal_Length_bucket",
                splits     = c(0, 3, 4.8, 8)) %>%
  select(Petal_Width, Petal_Length, Petal_Length_bucket, Species) 

iris_tbl_bucketized 

In [ ]:
# transform our data set, and then partition into 'training', 'test'
iris_partitions <- iris_tbl_bucketized %>% 
sdf_random_split(training = 0.9, test = 0.2, seed = 1099) #sample(.Machine$integer.max, 1)
#iris_partitions

# fit a linear model to the training dataset
iris_fit <- iris_partitions$training %>% 
ml_linear_regression(response = "Petal_Length_bucket", features = c("Petal_Width"))  
#iris_fit
summary(iris_fit)

In [ ]:
ml_predict(iris_fit, iris_tbl_bucketized) 


In [ ]:
# predict Petal_Width
pw_predicted <- ml_predict(iris_fit, iris_tbl_bucketized) %>% 
 ft_bucketizer(input_col  = "prediction",
                output_col = "prediction_bucket",
                splits     = c(-0.5, 0.5, 1.5, 2.5)) %>%
 collect
#pw_predicted
table(pw_predicted$Petal_Length_bucket, pw_predicted$prediction_bucket)

In [ ]:
pw_predicted %>% filter(Petal_Length_bucket != prediction_bucket) %>% 
arrange(desc(prediction))

In [ ]:
pw_predicted %>% filter(Petal_Length_bucket != prediction_bucket)

In [ ]:
### Clasificación con árbol

In [ ]:
dt_model <- iris_partitions$training %>%
  ml_decision_tree(Species ~ Petal_Length + Petal_Width, type = "classification")
dt_model

In [ ]:
dt_predict <- ml_predict(dt_model, iris_partitions$test) %>%
  collect

table(dt_predict$Species, dt_predict$prediction)

### Clasificación con Random Forest

In [ ]:
rf_model <- iris_partitions$training %>%
  ml_random_forest(Species ~ Petal_Length + Petal_Width, type = "classification")
rf_model

In [ ]:
rf_predict <- ml_predict(rf_model, iris_partitions$test) %>%
  collect

table(rf_predict$Species, rf_predict$prediction)

# # # Análisis exploratorio de datos con mtcars

Realizarmos un análisis exploratorio de los datos y después una regresión linear.

In [ ]:
#mtcars
head(mtcars)

In [ ]:
summary(mtcars)

In [ ]:
mtcars %>% count(cyl) 

mtcars %>% group_by(cyl) %>% 
   summarise(mean_of_weight = mean(wt),.groups = 'drop')

In [ ]:
mtcars %>% group_by(gear) %>% 
  summarise(mean_mpg_by_gear = mean(mpg),.groups = 'drop')

In [ ]:
mtcars %>% filter(hp == max(hp)) 
mtcars %>% filter(hp == min(hp)) 

In [ ]:
mtcars %>% filter(disp == max(disp)) 
mtcars %>% filter(disp == min(disp)) 

In [ ]:
# De todos los coches con 4 gears y >=4 cyl cuál tiene el máximo displ y el mínimo hp?
mtcars %>% 
filter((gear==4)& (cyl>=4)) -> mycars_resultado_intermedio

mycars_resultado_intermedio %>% filter((disp==max(disp)))
mycars_resultado_intermedio %>% filter((hp==min(hp)))


# # # Ejemplo de regresión linear con mtcars

In [ ]:
ggplot(mtcars, aes(x = wt, y = mpg)) + 
  geom_point() +
  stat_smooth(method = "lm", col = "red")

In [ ]:
ggplot(mtcars, aes(x = cyl, y = mpg)) + 
  geom_point() +
  stat_smooth(method = "lm", col = "red")

In [ ]:
# A continuación veremos un ejemplo que hace uso de ml_linear_regression para ajustar un modelo de regresión lineal. Se hará uso del dataset mtcars (que viene de serie) y veremos si podemos predecir el consumo de combustible (**mpg**, miles per galon) basándonos en el peso (**wt**) y el número de cilindros (**cyl**) del motor. Para ello vamos a asumir que la relación entre **mpg** y el resto de variables es lineal.

# copy mtcars into spark
mtcars_tbl <- copy_to(sc, mtcars, overwrite="TRUE")

In [ ]:
mtcars_tbl

In [ ]:
summary(mtcars)

In [ ]:
# transform our data set, and then partition into 'training', 'test'
mtcars_partitions <- mtcars_tbl %>%
  filter(hp >= 30) %>%
  mutate(cyl8 = cyl == 8) %>%
  ft_bucketizer(input_col  = "mpg",
                output_col = "mpg_bucket",
                splits     = c(10, 16, 22, 45)) %>%
  sdf_random_split(training = 0.8, test = 0.2, seed = 399) #sample(.Machine$integer.max, 1)
#partitions

In [ ]:
mtcars_partitions$training 


In [ ]:

# fit a linear model to the training dataset
mtcars_fit <- mtcars_partitions$training %>%
  ml_linear_regression(mpg ~ wt+cyl)
mtcars_fit

#ml_linear_regression(response = "mpg", features = c("wt", "cyl"))


In [ ]:
ml_predict(mtcars_fit, mtcars_partitions$test) %>% 
ft_bucketizer(input_col  = "prediction",
                output_col = "prediction_bucket",
                splits     = c(6, 17, 22, 45)) %>%
select(mpg,wt,cyl,prediction, mpg_bucket, prediction_bucket) %>%
filter(mpg_bucket != prediction_bucket)

75% accuracy 15/20 with the test dataset

In [ ]:
mtcars_partitions$test %>% collect

In [ ]:
summary(mtcars_fit)

In [ ]:
mtcars_partitions

# # Actividad propuesta:

¿Puedes analizar mejorar el modelo de regresión linear obtenido con el conjunto de test?

In [ ]:
mpg_predicted <- ml_predict(mtcars_fit, mtcars_partitions$test)
mpg_predicted %>% select(mpg, mpg_bucket, cyl, wt, prediction)

In [ ]:
# predict mpg of mtcars
mpg_predicted <- ml_predict(mtcars_fit, mtcars_partitions$test) %>% 
 ft_bucketizer(input_col  = "prediction",
                output_col = "prediction_bucket",
                splits     = c(5, 12, 17, 22, 45)) %>%
 collect

#mpg_predicted
table(mpg_predicted$mpg_bucket, mpg_predicted$prediction_bucket)

In [ ]:
mpg_predicted

# # Ejemplo de Naive Bayes

In [ ]:
# Fit a naive bayes model 
system.time(
  nb_model <- mtcars_partitions$training %>% ml_naive_bayes(cyl8 ~ .)
)

In [ ]:
pred <- ml_predict(nb_model, mtcars_partitions$test)

In [ ]:
#pred
ml_multiclass_classification_evaluator(pred)

In [ ]:
pred %>% select(cyl8, prediction, probability_0)
#pred

In [ ]:
#spark_disconnect(sc)
